In [ ]:
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.model_selection import KFold, cross_val_score
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from keras.preprocessing import image
import matplotlib.pyplot as plt
from keras.models import Model
from keras import applications
from sklearn import metrics
from random import randint
import tensorflow as tf
from sklearn import svm
%matplotlib inline 
import warnings
import seaborn as sns
import itertools
import pandas as pd
import numpy as np





In [ ]:



def split(df,label):
    X_train, X_test, Y_train, Y_test = train_test_split(df, label, test_size=0.25, random_state=42)
    return X_train, X_test, Y_train, Y_test



classifiers = [ 'SVM', 
                'RandomForest', 
               'DecisionTree', 
               'KNeighbors']

models = [svm.SVC(kernel='linear'),
          RandomForestClassifier(n_estimators=200, random_state=0),
          DecisionTreeClassifier(random_state=0),
          KNeighborsClassifier()]


    


In [ ]:
def initilization_of_population(size,n_feat):
    population = []
    for i in range(size):
        chromosome = np.ones(n_feat,dtype=np.bool)     
        chromosome[:int(0.3*n_feat)]=False             
        np.random.shuffle(chromosome)
        population.append(chromosome)
    return population


def fitness_score(population):
    scores = []
    for chromosome in population:
        logmodel2.fit(X_train.iloc[:,chromosome],Y_train)         
        predictions = logmodel2.predict(X_test.iloc[:,chromosome])
        scores.append(accuracy_score(Y_test,predictions))
    scores, population = np.array(scores), np.array(population) 
    inds = np.argsort(scores)                                    
    return list(scores[inds][::-1]), list(population[inds,:][::-1]) 


def selection(pop_after_fit,n_parents):
    population_nextgen = []
    for i in range(n_parents):
        population_nextgen.append(pop_after_fit[i])
    return population_nextgen


def crossover(pop_after_sel):
    pop_nextgen = pop_after_sel
    for i in range(0,len(pop_after_sel),2):
        new_par = []
        child_1 , child_2 = pop_nextgen[i] , pop_nextgen[i+1]
        new_par = np.concatenate((child_1[:len(child_1)//2],child_2[len(child_1)//2:]))
        pop_nextgen.append(new_par)
    return pop_nextgen


def mutation(pop_after_cross,mutation_rate,n_feat):   
    mutation_range = int(mutation_rate*n_feat)
    pop_next_gen = []
    for n in range(0,len(pop_after_cross)):
        chromo = pop_after_cross[n]
        rand_posi = [] 
        for i in range(0,mutation_range):
            pos = randint(0,n_feat-1)
            rand_posi.append(pos)
        for j in rand_posi:
            chromo[j] = not chromo[j]  
        pop_next_gen.append(chromo)
    return pop_next_gen

def generations(df,label,size,n_feat,n_parents,mutation_rate,n_gen,X_train,
                                   X_test, Y_train, Y_test):
    best_chromo= []
    best_score= []
    population_nextgen=initilization_of_population(size,n_feat)
    for i in range(n_gen):
        scores, pop_after_fit = fitness_score(population_nextgen)
        print('Best score in generation',i+1,':',scores[:1])  #2
        pop_after_sel = selection(pop_after_fit,n_parents)
        pop_after_cross = crossover(pop_after_sel)
        population_nextgen = mutation(pop_after_cross,mutation_rate,n_feat)
        best_chromo.append(pop_after_fit[0])
        best_score.append(scores[0])
    return best_chromo,best_score

  

In [ ]:
data_bc = pd.read_csv("/content/drive/MyDrive/Collapsed lung /features.csv")
label_bc = data_bc["class"]
data_bc.drop(["id", "class"],axis = 1,inplace = True)
#500_features

logmodel2 = RandomForestClassifier(n_estimators=200, random_state=0)
X_train,X_test, Y_train, Y_test = split(data_bc,label_bc)
chromo_df_bc,score_bc=generations(data_bc,label_bc,size=80,n_feat=data_bc.shape[1],n_parents=65,mutation_rate=0.20,n_gen=50,
                         X_train = X_train,X_test = X_test,Y_train = Y_train,Y_test = Y_test)

In [ ]:
Y_pred = logmodel2.predict(X_test)
y_pred = np.argmax(Y_pred, axis=1)

report = classification_report(test_set.classes, y_pred, output_dict=True)
dff = pd.DataFrame(report).transpose()
dff.to_excel("/content/drive/MyDrive/Collapsed lung /features.xlsx")